<a href="https://colab.research.google.com/github/antonioalbanese/Time-Series-Anomaly-Detection-An-experimental-survey/blob/main/Experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch_lightning torchinfo --quiet

     |████████████████████████████████| 796 kB 26.6 MB/s 
     |████████████████████████████████| 136 kB 61.3 MB/s 
     |████████████████████████████████| 529 kB 73.5 MB/s 
     |████████████████████████████████| 87 kB 6.8 MB/s 


In [2]:
!git clone https://github.com/antonioalbanese/Time-Series-Anomaly-Detection-An-experimental-survey/

%cd Time-Series-Anomaly-Detection-An-experimental-survey

%mkdir dataset
!gdown https://drive.google.com/uc?id=1ZCLBU_pKTbsPlcj_LwxZE3IRy6mrlys3 -O="./dataset/MSL.zip"
!gdown https://drive.google.com/uc?id=18JNYBsaX7tu0Qfgo92nCBCklv471L1xB -O="./dataset/SMD.zip"
import zipfile
with zipfile.ZipFile("./dataset/MSL.zip", 'r') as zip_ref:
    zip_ref.extractall("./dataset/")
with zipfile.ZipFile("./dataset/SMD.zip", 'r') as zip_ref:
    zip_ref.extractall("./dataset/")

Cloning into 'Time-Series-Anomaly-Detection-An-experimental-survey'...
remote: Enumerating objects: 522, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 522 (delta 0), reused 0 (delta 0), pack-reused 517
Receiving objects: 100% (522/522), 2.68 MiB | 19.05 MiB/s, done.
Resolving deltas: 100% (273/273), done.
/content/Time-Series-Anomaly-Detection-An-experimental-survey
Downloading...
From: https://drive.google.com/uc?id=1ZCLBU_pKTbsPlcj_LwxZE3IRy6mrlys3
To: /content/Time-Series-Anomaly-Detection-An-experimental-survey/dataset/MSL.zip
100% 767k/767k [00:00<00:00, 163MB/s]
Downloading...
From: https://drive.google.com/uc?id=18JNYBsaX7tu0Qfgo92nCBCklv471L1xB
To: /content/Time-Series-Anomaly-Detection-An-experimental-survey/dataset/SMD.zip
100% 195M/195M [00:02<00:00, 81.3MB/s]


In [1]:
!mkdir input
#normal period
!python USAD/gdrivedl.py https://drive.google.com/open?id=1rVJ5ry5GG-ZZi5yI4x9lICB8VhErXwCw input/
#anomalies
!python USAD/gdrivedl.py https://drive.google.com/open?id=1iDYc0OEmidN712fquOBRFjln90SbpaE7 input/

python3: can't open file 'USAD/gdrivedl.py': [Errno 2] No such file or directory
python3: can't open file 'USAD/gdrivedl.py': [Errno 2] No such file or directory


In [ ]:
### USAD
from AnomalyDetectionMethodClass import ADMethod
settings={
    'EPOCHS':20,
    'BATCH_SIZE': 7919,
    'SEQ_LEN': 12,
    'HIDDEN_SIZE': 100,
    'ALPHA': 0.5,
    'BETA': 0.5,
    'CONFIDENCE': 0.82,
    'DATASET': 'SWAT'}
data_path = ''
method = ADMethod('USAD',settings,'')

method.prepare_pipeline()
h = method.train()
scores, report = method.test()

In [ ]:
###  TRANSFORMER
from AnomalyDetectionMethodClass import ADMethod
data_path = './dataset/MSL'
settings = {
    'lr':1e-4,
    'num_epochs':3,
    'k':3,
    'win_size':100,
    'input_c':55,
    'output_c':55,
    'batch_size':256,
    'pretrained_model':None,
    'dataset':'MSL',
    'mode':'train', # choices=['train', 'test']
    'data_path':'./dataset/MSL',
    'model_save_path':'checkpoints',
    'anormly_ratio':1
}
method = ADMethod('transformer',settings, data_path)

method.prepare_pipeline()
method.train()
scores, report = method.test()

In [ ]:
### DEEPANT
from AnomalyDetectionMethodClass import ADMethod
data_path = './NAB/realKnownCause/ambient_temperature_system_failure.csv'
settings = {
  'SEQ_LEN': 10,
  'out_dim': 1,
  'EPOCHS': 50,
  'LR': 0.0001,
  'CONFIDENCE': 0.80
}
method = ADMethod('DeepAnt',settings, data_path)

method.prepare_pipeline()
l = method.train()
scores, report = method.test()
# method.results()

# import matplotlib.pyplot as plt
# import seaborn as sns
# def plot_predictions(preds, threshold, bins = 80):
#     sns.displot(preds, bins = bins, kde = True, height = 8, aspect = 2)
#     plt.axvline(x = threshold, color = 'r', linestyle = '--', label = 'Chosen Threshold')
#     plt.title('Loss Distribution')
#     plt.legend()

# def loss_plot(preds, threshold):
#     fig = plt.figure(figsize=(15,8))
#     plt.plot(preds, label = 'Loss')
#     # preds.plot(figsize = (15,8), title = 'Chosen Threshold', label = 'Loss')
#     plt.axhline(y = threshold, color = 'r', linestyle = '--', label = 'Chosen Threshold')
#     plt.legend()
#     plt.show()

# def ts_plot(df, preds, threshold, alg = 'DeepAnT', range = None):
#     idx = preds.loc[lambda x: x > threshold].index
#     plt.figure(figsize = (15,8))
#     if range is not None:
#         min, max = range
#         df = df[min:max]
#     plt.plot(df, label = '_nolegend_')
#     plt.scatter(idx, df.loc[idx], color = 'red', label = 'Detected Anomalies by DeepAnt')
#     plt.title('Detected Anomalies in Time Series')
#     plt.legend()
#     plt.show()



# import sklearn
# import numpy as np
# sc = sklearn.preprocessing.MinMaxScaler(feature_range=(0,1))
# losses = sc.fit_transform(np.array(lp).reshape(-1, 1))
# plot_predictions(losses, 0.80)
# loss_plot(losses, 0.80)


# import pandas as pd
# df = pd.read_csv(data_path, index_col = 'timestamp', parse_dates=['timestamp'])
# target_idx = method.solver.dataset.timestamp
# preds_losses = pd.Series(losses.reshape((-1)), index = target_idx)
# ts_plot(df, preds_losses, 0.80)

In [ ]:
!pip install torchinfo
from torchinfo import summary
summary(method.solver.model)